<a href="https://colab.research.google.com/github/StephinAntonio7/google-docs-api-integration/blob/main/Ansible_Health_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 54.9 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.164.0
    Uninstalling google-api-python-client-2.164.0:
      Successfully uninstalled google-api-python-client-2.164.0


In [18]:
from googleapiclient.discovery import build
from google.oauth2 import service_account
import re

# Define API Scope and authenticate
SCOPES = [
    'https://www.googleapis.com/auth/documents',
    'https://www.googleapis.com/auth/drive'
]
SERVICE_ACCOUNT_FILE = '/content/service_account.json'  # Update path if necessary

# Authenticate using the service account
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Build Google Docs and Drive services
docs_service = build('docs', 'v1', credentials=creds)
drive_service = build('drive', 'v3', credentials=creds)

# Document content
markdown_notes = """
# Product Team Sync - May 15, 2023

## Attendees
Sarah Chen (Product Lead)
Mike Johnson (Engineering)
Anna Smith (Design)
David Park (QA)

## Agenda

### 1. Sprint Review
Completed Features:
  - User authentication flow
  - Dashboard redesign
  - Performance optimization:
    - Reduced load time by 40%
    - Implemented caching solution
Pending Items:
  - Mobile responsive fixes
  - Beta testing feedback integration

### 2. Current Challenges
- Resource constraints in QA team
- Third-party API integration delays
- User feedback on new UI:
  - Navigation confusion
  - Color contrast issues

### 3. Next Sprint Planning
Priority Features:
  - Payment gateway integration
  - User profile enhancement
  - Analytics dashboard
Technical Debt:
  - Code refactoring
  - Documentation updates

## Action Items
@sarah: Finalize Q3 roadmap by Friday
@mike: Schedule technical review for payment integration
@anna: Share updated design system documentation
@david: Prepare QA resource allocation proposal

## Next Steps
- Schedule individual team reviews
- Update sprint board
- Share meeting summary with stakeholders

## Notes
- Next sync scheduled for May 22, 2023
- Platform demo for stakeholders on May 25
- Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

def create_google_doc(title, content):
    """
    Creates a Google Doc, inserts content, and shares it with a specific email.
    """
    try:
        # Create a new Google Doc
        doc = docs_service.documents().create(body={'title': title}).execute()
        doc_id = doc['documentId']

        # Initialize request list for inserting formatted content
        requests = []
        index = 1  # Start index for inserting content

        # Split content into lines for individual processing
        lines = content.split('\n')

        for line in lines:
            # Main title ("Product Team Sync") should be Heading 1 style
            if line.startswith('# '):
                requests.append({
                    'insertText': {'location': {'index': index}, 'text': line[2:] + '\n'}
                })
                index += len(line[2:]) + 1
                requests.append({
                    'updateParagraphStyle': {
                        'range': {'startIndex': index - len(line[2:]) - 1, 'endIndex': index},
                        'paragraphStyle': {'namedStyleType': 'HEADING_1'},
                        'fields': '*'  # Specify fields to be updated
                    }
                })
            # Section headers ("Attendees", "Agenda") should be Heading 2 style
            elif line.startswith('## '):
                requests.append({
                    'insertText': {'location': {'index': index}, 'text': line[3:] + '\n'}
                })
                index += len(line[3:]) + 1
                requests.append({
                    'updateParagraphStyle': {
                        'range': {'startIndex': index - len(line[3:]) - 1, 'endIndex': index},
                        'paragraphStyle': {'namedStyleType': 'HEADING_2'},
                        'fields': '*'  # Specify fields to be updated
                    }
                })
            # Sub-section headers (under Agenda) should be Heading 3 style
            elif line.startswith('### '):
                requests.append({
                    'insertText': {'location': {'index': index}, 'text': line[4:] + '\n'}
                })
                index += len(line[4:]) + 1
                requests.append({
                    'updateParagraphStyle': {
                        'range': {'startIndex': index - len(line[4:]) - 1, 'endIndex': index},
                        'paragraphStyle': {'namedStyleType': 'HEADING_3'},
                        'fields': '*'  # Specify fields to be updated
                    }
                })
            # Assignee mentions (@name) styling, add checkbox before assignee
            elif '@' in line:
                start_pos = line.find('@')
                assignee_match = re.search(r'@\w+', line)
                if assignee_match:
                    assignee = assignee_match.group()
                    # Add a checkbox before the assignee mention
                    checkbox = "☐ "  # Unicode for an empty checkbox symbol
                    line_with_checkbox = checkbox + line

                    end_pos = start_pos + len(assignee)

                    requests.append({'insertText': {'location': {'index': index}, 'text': line_with_checkbox + '\n'}})
                    index += len(line_with_checkbox) + 1
                    # Update text style for assignee mention
                    requests.append({
                        'updateTextStyle': {
                            'range': {
                                'startIndex': index - len(line_with_checkbox) - 1 + start_pos + 2,  # Skip checkbox space
                                'endIndex': index - len(line_with_checkbox) - 1 + end_pos + 2  # Skip checkbox space
                            },
                            'textStyle': {
                                'bold': True,
                                'foregroundColor': {
                                    'color': {
                                        'rgbColor': {
                                            'red': 0.2,
                                            'green': 0.6,
                                            'blue': 1.0
                                        }
                                    }
                                }
                            },
                            'fields': '*'  # Specify that all fields should be updated
                        }
                    })
            # Footer information (Meeting recorded by, Duration) in distinct style
            elif 'Meeting recorded by:' in line or 'Duration:' in line:
                requests.append({'insertText': {'location': {'index': index}, 'text': line + '\n'}})
                index += len(line) + 1
                # Apply distinct styling (e.g., italic and different font color)
                requests.append({
                    'updateTextStyle': {
                        'range': {'startIndex': index - len(line) - 1, 'endIndex': index},
                        'textStyle': {
                            'italic': True,
                            'foregroundColor': {
                                'color': {
                                    'rgbColor': {
                                        'red': 0.8,
                                        'green': 0.8,
                                        'blue': 0.8
                                    }
                                }
                            }
                        },
                        'fields': '*'  # Specify that all fields should be updated
                    }
                })
            # Action items with checkbox (Unicode checkbox)
            elif '☐' in line or '☑' in line:
                requests.append({'insertText': {'location': {'index': index}, 'text': line + '\n'}})
                index += len(line) + 1
            # Regular content (non-header, non-action items)
            else:
                requests.append({'insertText': {'location': {'index': index}, 'text': line + '\n'}})
                index += len(line) + 1

        # Execute batch update to insert formatted content
        docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

        # --- SHARE DOCUMENT WITH YOUR GOOGLE ACCOUNT ---
        user_permission = {
            'type': 'user',
            'role': 'writer',  # Can be 'reader' or 'writer'
            'emailAddress': 'stephin.roberts@gmail.com'
        }

        # Add permission
        drive_service.permissions().create(
            fileId=doc_id,
            body=user_permission,
            fields='id'
        ).execute()

        return f"Document created and shared: https://docs.google.com/document/d/{doc_id}"

    except Exception as e:
        return f"Error creating Google Doc: {str(e)}"

# Create document and print the URL
doc_url = create_google_doc('Product Team Sync - May 15, 2023', markdown_notes)
print(doc_url)



Document created and shared: https://docs.google.com/document/d/15rbquv8oWZpWJH8QWD9H06vLQeLCq4hYG0z6spSae9k
